UNIVERSIDADE FEDERAL DO PARANÁ <br>
CE329 - APRENDIZAGEM DE MAQUINA <br>
Profº: Eduardo Vargas Ferreira <br>
Alunos:  GRR20233876 – Leonardo Gonçalves Fischer <br>
         GRR20233881 – Raymundo Eduardo Pilz <br>


# ANÁLISE DE CHURN EM UMA INSTITUIÇÃO FINANCEIRA

## PROBLEMÁTICA:
O problema a ser abordado neste trabalho é a taxa de churn em uma instituição financeira. A taxa 
de churn, que se refere à perda de clientes ao longo do tempo, é uma métrica crítica para empresas 
de serviços financeiros, pois afeta diretamente a receita e a satisfação do cliente. O objetivo deste 
estudo é analisar e compreender os fatores que podem estar contribuindo para o churn de clientes 
e identificar estratégias para reduzi-lo.
DESCRIÇÃO DO PROBLEMA:
Conhecer a taxa de churn de qualquer instituição é importante visto que para a empresa custa muito 
mais captar um novo cliente do que manter um que já possui.
Compreender esses fatores é fundamental para tomar decisões informadas que permite a 
companhia desenvolver programas de leadade e implementar estratégias que aumentem a retenção 
de clientes.
## OBJETIVOS:
* Análise exploratória e descritiva dos principais indicadores e características dos clientes, tais 
como:
    * Pontuação de crédito;
    * Localização;
    * Gênero;
    * Idade;
    * Tempo de permanência;
    * Saldo da conta;
    * Número de produtos;
    * Uso de cartão de crédito
    * Atividade de conta;
    * Salário estimado;
    * Churn;
    * Reclamações realizadas pelo cliente;
    * Índice de satisfação do cliente;
    * Tipo de cartão;
    * Pontos Acumulados.
* Identificação e correção de possíveis inconsistências nos dados fornecidos;
* Identificar quais dessas características têm maior correlação com o churn de clientes;
* Modelos de predições dos dados para explorar padrões e tendências nos dados relacionados 
ao churn;
* Comparações com modelos com todos os resultados gerados;
* Propor recomendações estratégicas com base nos insights obtidos para reduzir a taxa de churn.

## dataset disponivel em: 
https://www.kaggle.com/datasets/radheshyamkollipara/bank-customer-churn

In [14]:
# carregando as bibliotecas
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(color_codes=True)
import warnings
warnings.filterwarnings('ignore')

In [7]:
## Carregando os dados

data = pd.read_csv('dataset\\Customer-Churn-Records.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1,1,2,DIAMOND,464
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,1,3,DIAMOND,456
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,1,3,DIAMOND,377
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,5,GOLD,350
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,0,5,GOLD,425


In [21]:
# Verificar colunas com dados faltantes
data_na = (data.isnull().sum() / len(data)) * 100
data_na = data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)[:30]

# Verificar se há colunas com valores ausentes
if not data_na.empty:
    # Criar um gráfico de barras
    plt.figure(figsize=(10, 5))
    sns.barplot(x=data_na.index, y=data_na)
    plt.xticks(rotation=90)
    plt.xlabel('Features')
    plt.ylabel('Percentual de valores missing')
    plt.title('Percentual de dados faltantes por variável')
    plt.grid(True)
    plt.show()
else:
    print("Não há valores ausentes nas colunas selecionadas.")

Não há valores ausentes nas colunas selecionadas.


In [25]:
# Excluindo colunas desnecessarias
data = data.drop(['Surname'], axis=1)
data.head()

,RowNumber,CustomerId,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain,Satisfaction Score,Card Type,Point Earned
0,1,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1,1,2,DIAMOND,464
1,2,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,1,3,DIAMOND,456
2,3,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,1,3,DIAMOND,377
3,4,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,5,GOLD,350
4,5,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,0,5,GOLD,425


In [27]:
# Alterando nome das Colunas
data = data.rename(columns={"RowNumber": "Índice",
                            "CustomerId": "ID",
                            "CreditScore": "Pontuação de Crédito", 
                            "Geography": "Localização", 
                            "Gender": "Gênero",
                            "Age": "Idade",
                            "Tenure": "Tempo de Permanência",
                            "Balance": "Saldo",
                            "NumOfProducts": "Produtos",
                            "HasCrCard": "Possui Cartão de Crédito",
                            "IsActiveMember": "Membro Ativo",
                            "EstimatedSalary": "Salário Estimado",
                            "Exited": "Churn",
                            "Complain": "Reclamações",
                            "Satisfaction Score": "Índice de Satisfação",
                            "Card Type": "Tipo de Cartão",
                            "Point Earned": "Pontos Acumulados"})
data.head()

,Índice,ID,Pontuação de Crédito,Localização,Gênero,Idade,Tempo de Permanência,Saldo,Produtos,Possui Cartão de Crédito,Membro Ativo,Salário Estimado,Churn,Reclamações,Índice de Satisfação,Tipo de Cartão,Pontos Acumulados
0,1,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1,1,2,DIAMOND,464
1,2,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,1,3,DIAMOND,456
2,3,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,1,3,DIAMOND,377
3,4,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,5,GOLD,350
4,5,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,0,5,GOLD,425


In [28]:
# Verificando o volume de dados do dataset para depois verificar se tem linhas duplicadas
data.shape

(10000, 17)

In [30]:
# Verificando linhas duplicadas
duplicate_rows_data = data[data.duplicated()]
print("Número de linhas duplicadas: ", duplicate_rows_data.shape)

Número de linhas duplicadas:  (0, 17)


In [31]:
# Caso houvessem linhas duplicadas, seriam excluidas e impresso o novo dataset
data = data.drop_duplicates()
data.head()

,Índice,ID,Pontuação de Crédito,Localização,Gênero,Idade,Tempo de Permanência,Saldo,Produtos,Possui Cartão de Crédito,Membro Ativo,Salário Estimado,Churn,Reclamações,Índice de Satisfação,Tipo de Cartão,Pontos Acumulados
0,1,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1,1,2,DIAMOND,464
1,2,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,1,3,DIAMOND,456
2,3,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,1,3,DIAMOND,377
3,4,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,5,GOLD,350
4,5,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,0,5,GOLD,425


In [26]:
# Obter informações gerais do conjunto de dados
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   RowNumber           10000 non-null  int64  
 1   CustomerId          10000 non-null  int64  
 2   CreditScore         10000 non-null  int64  
 3   Geography           10000 non-null  object 
 4   Gender              10000 non-null  object 
 5   Age                 10000 non-null  int64  
 6   Tenure              10000 non-null  int64  
 7   Balance             10000 non-null  float64
 8   NumOfProducts       10000 non-null  int64  
 9   HasCrCard           10000 non-null  int64  
 10  IsActiveMember      10000 non-null  int64  
 11  EstimatedSalary     10000 non-null  float64
 12  Exited              10000 non-null  int64  
 13  Complain            10000 non-null  int64  
 14  Satisfaction Score  10000 non-null  int64  
 15  Card Type           10000 non-null  object 
 16  Point